# Revised case normalization for Hirslanden Zurich 2018

Notes:
- There is 'n.ü.' in the duration_of_stay, at the moment repöace them with nan



In [ ]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os
import numpy as np
from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

ModuleNotFoundError: No module named 'py'

In [ ]:

file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
        'Hirslanden Zurich',
         '2018',
        ['Änderungen_Hirslanden_2018'])

print(file_info)


In [ ]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

try:
    df_revised_case_d2d = normalize(file_info, 0)
except ValueError:
    print(str(sys.exc_info()))

In [ ]:
df_revised_case_d2d

# Match revised DtoD data with the bfs data from the database


In [ ]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

In [ ]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

In [ ]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [ ]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

In [ ]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

In [ ]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [ ]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

In [ ]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
assert len(df_revised_case_normalized.columns) == len(COLS_TO_OUTPUT)

df_revised_case_normalized.head()

# Export the normalized revised cases with the neccessary columns to a csv file


In [ ]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [ ]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

In [ ]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))